### 일반적임 똑같음 
절차과정 : nan값 가지고있는 피쳐링에 각각 mean 을 넣어보기도함 (test낮음)
그래서 결국  nan값 -999 baaseline 과같이 동일하게함
baseline 이랑 동일(피쳐링개수빼고)
내 생각: 그냥 baseline 에있는 피쳐들 따라가면서 board 

In [ ]:
import os
from typing import List, Dict
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
import lightgbm as lgb
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
# 파일 호출
data_path: str = "data"
train_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "train.csv")).assign(_type="train") # train 에는 _type = train 
test_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv")).assign(_type="test") # test 에는 _type = test
submission_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv")) # ID, target 열만 가진 데이터 미리 호출
df: pd.DataFrame = pd.concat([train_df, test_df], axis=0)
df


,ID,target,_type
0,2023-01-01 00:00:00,2.0,train
1,2023-01-01 01:00:00,1.0,train
2,2023-01-01 02:00:00,1.0,train
3,2023-01-01 03:00:00,1.0,train
4,2023-01-01 04:00:00,2.0,train
...,...,...,...
2787,2024-04-26 03:00:00,NaN,test
2788,2024-04-26 04:00:00,NaN,test
2789,2024-04-26 05:00:00,NaN,test
2790,2024-04-26 06:00:00,NaN,test


In [3]:
file_names : List[str]=[
    f for f in os.listdir(data_path) if f.startswith('HOURLY_') and f.endswith(".csv")
]

file_dict: Dict[str,pd.DataFrame]={
    f.replace(".csv",""): pd.read_csv(os.path.join(data_path,f)) for f in file_names
}

for _file_name, _df in tqdm(file_dict.items()):
    _rename_rule={
        col: f"{_file_name.lower()}_{col.lower()}" if col!="datetime" else "ID"
        for col in _df.columns
    }
    _df=_df.rename(_rename_rule,axis=1)
    df=df.merge(_df,on="ID",how="left")


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 107/107 [00:01<00:00, 77.75it/s]


In [4]:
df
missing_values=df.isnull().sum()
missing_values
missing_percentage= (missing_values)/len(df) *100
missing_percentage

sorted_missing_percentage=missing_percentage.sort_values(ascending=False)

# null 100 percent인 feature들
name_null=[]
for col, val in sorted_missing_percentage.items():
    if val==100:
        name_null.append(col)        
len(name_null)
for i in name_null:
    df=df.drop(columns=i)
df.shape


(11552, 215)

In [ ]:
#내일 베이스라인의 선택적인 컬럼만 가져와서 nan값에 넣고 해보기

In [5]:
cols_dict : Dict[str,str] = {}
for i in df.columns:
    
    parts=i.split("_")
    if len(parts)>=3:
        cols_dict[i]="_".join(parts[2:])
    else:
        cols_dict[i]=i

cols_dict2: Dict[str, str] = {
    "ID": "ID",
    "target": "target",
    "_type": "_type",
    "coinbase-premium-index_coinbase_premium_gap": "coinbase_premium_gap",
    "coinbase-premium-index_coinbase_premium_index": "coinbase_premium_index",
    "funding-rates_all_exchange_funding_rates": "funding_rates",
    "liquidations_all_exchange_all_symbol_long_liquidations": "long_liquidations",
    "liquidations_all_exchange_all_symbol_long_liquidations_usd": "long_liquidations_usd",
    "liquidations_all_exchange_all_symbol_short_liquidations": "short_liquidations",
    "liquidations_all_exchange_all_symbol_short_liquidations_usd": "short_liquidations_usd",
    "open-interest_all_exchange_all_symbol_open_interest": "open_interest",
    "taker-buy-sell-stats_all_exchange_taker_buy_ratio": "buy_ratio",
    "taker-buy-sell-stats_all_exchange_taker_buy_sell_ratio": "buy_sell_ratio",
    "taker-buy-sell-stats_all_exchange_taker_buy_volume": "buy_volume",
    "taker-buy-sell-stats_all_exchange_taker_sell_ratio": "sell_ratio",
    "taker-buy-sell-stats_all_exchange_taker_sell_volume": "sell_volume",
    "addresses-count_addresses_count_active": "active_count",
    "addresses-count_addresses_count_receiver": "receiver_count",
    "addresses-count_addresses_count_sender": "sender_count",
}
cols_dict
df = df[cols_dict.keys()].rename(cols_dict,axis=1)

In [6]:
for i in cols_dict2.keys():
    df = df.rename(columns={i: cols_dict2[i]})


In [7]:
df.columns.nunique()

215

In [ ]:
# # Nan 값 채우기
# eda_df=df.loc[df["_type"]=="train"]
# eda_test=df.loc[df["_type"]=="test"]
# missing_values = eda_df.isnull().sum()
# missing_percentage = (missing_values / len(eda_df)) * 100
# sorted_missing_percentage = missing_percentage.sort_values(ascending=False)

# # Identify columns with missing values
# null = [col for col in eda_df.columns if missing_values[col] != 0]

# # Fill missing values with the mean of each column
# for col in null:
#     eda_df.loc[:, col] = eda_df[col].fillna(eda_df[col].mean())
    
# missing_values2 = eda_test.isnull().sum()
# missing_percentage2 = (missing_values2 / len(eda_test)) * 100
# sorted_missing_percentage2 = missing_percentage2.sort_values(ascending=False)

# null_test=[col for col in eda_test.columns if missing_values2[col] !=0 and col!='target']

# for col in null_test:
#     eda_test.loc[:,col] = eda_test[col].fillna(eda_df[col].mean())

# df=pd.concat([eda_df,eda_test],axis=0)
# df

In [8]:
df = df.assign(
    liquidation_diff=df["long_liquidations"]-df["short_liquidations"],
    liquidation_usd_diff= df['long_liquidations_usd']-df['short_liquidations_usd'],
    volume_diff=df["buy_volume"] - df["sell_volume"],
    liquidation_diffg=np.sign(df["long_liquidations"]-df["short_liquidations"]),
    liquidation_usd_diffg=np.sign(df['long_liquidations_usd']-df['short_liquidations_usd']),
    volume_diffg=np.sign(df["buy_volume"] - df["sell_volume"]),
    buy_sell_volume_ratio=df["buy_volume"] / (df["sell_volume"] + 1)
)

In [ ]:
# category_cols : List[str] = ["liquidation_diffg", "liquidation_usd_diffg", "volume_diffg"]
# conti_cols: List[str] = [ i for i in df.columns 
#                          if i not in ["ID","target","_type","liquidation_diffg", "liquidation_usd_diffg", "volume_diffg"]]
# len(conti_cols)

In [ ]:
# def shift_feature(
#     df: pd.DataFrame,
#     conti_cols: List[str],
#     intervals: List[int],
# ) -> List[pd.Series]:
#     """
#     연속형 변수의 shift feature 생성
#     Args:
#         df (pd.DataFrame)
#         conti_cols (List[str]): continuous colnames
#         intervals (List[int]): shifted intervals
#     Return:
#         List[pd.Series]
#     """
#     df_shift_dict = [
#         df[conti_col].shift(interval).rename(f"{conti_col}_{interval}")
#         for conti_col in conti_cols
#         for interval in intervals
#     ]
#     return df_shift_dict

# # 최대 24시간의 shift 피쳐를 계산
# shift_list = shift_feature(
#     df=df, conti_cols=conti_cols, intervals=[_ for _ in range(1, 24)]
# )

In [9]:
# df = pd.concat([df, pd.concat(shift_list, axis=1)], axis=1)

# # 타겟 변수를 제외한 변수를 forwardfill, -999로 결측치 대체
_target = df["target"]
df = df.ffill().fillna(-999).assign(target = _target)

# _type에 따라 train, test 분리
train_df = df.loc[df["_type"]=="train"].drop(columns=["_type"])
test_df = df.loc[df["_type"]=="test"].drop(columns=["_type"])

In [15]:
x_train, x_valid, y_train, y_valid = train_test_split(
    train_df.drop(["target", "ID"], axis = 1), 
    train_df["target"].astype(int), 
    test_size=0.2,
    random_state=42,
)

# lgb dataset
train_data = lgb.Dataset(x_train, label=y_train)
valid_data = lgb.Dataset(x_valid, label=y_valid, reference=train_data)

# lgb params
params = {
    "boosting_type": "gbdt",
    "objective": "multiclass",
    "metric": "multi_logloss",
    "num_class": 4,
    "num_leaves": 50,
    "learning_rate": 0.05,
    "n_estimators": 30,
    "random_state": 42,
    "verbose": 0,
}

# lgb train
lgb_model = lgb.train(
    params=params,
    train_set=train_data,
    valid_sets=valid_data,
)

# lgb predict
y_valid_pred = lgb_model.predict(x_valid)
y_valid_pred_class = np.argmax(y_valid_pred, axis = 1)

# score check
accuracy = accuracy_score(y_valid, y_valid_pred_class)
auroc = roc_auc_score(y_valid, y_valid_pred, multi_class="ovr")

print(f"acc: {accuracy}, auroc: {auroc}")

/Users/sinseunghun/anaconda3/envs/my_proejct/lib/python3.9/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


acc: 0.4360730593607306, auroc: 0.6388032318060743


In [16]:
importance = lgb_model.feature_importance()
feature_names=train_df.drop(["target", "ID"], axis = 1).columns
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importance
})

# Sort by importance (optional)
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Display the DataFrame
print(feature_importance_df[:30])


                                               Feature  Importance
36       open-interest_bitfinex_btc_usdt_open_interest         149
139        price-ohlcv_all_exchange_spot_btc_usd_close         114
57         taker-buy-sell-stats_bybit_taker_buy_volume         113
33   open-interest_htx_global_all_symbol_open_interest         112
42      taker-buy-sell-stats_deribit_taker_sell_volume          99
81    taker-buy-sell-stats_htx_global_taker_buy_volume          95
175      taker-buy-sell-stats_bitmex_taker_sell_volume          93
193                funding-rates_deribit_funding_rates          89
143                      block-interval_block_interval          88
73         tokens-transferred_tokens_transferred_total          85
74          tokens-transferred_tokens_transferred_mean          83
165  open-interest_huobi_global_btc_usdt_open_interest          82
155           taker-buy-sell-stats_okx_taker_buy_ratio          81
54                                      receiver_count        

In [12]:
# performance 체크후 전체 학습 데이터로 다시 재학습
x_train = train_df.drop(["target", "ID"], axis = 1)
y_train = train_df["target"].astype(int)
train_data = lgb.Dataset(x_train, label=y_train)
lgb_model = lgb.train(
    params=params,
    train_set=train_data,
)

/Users/sinseunghun/anaconda3/envs/my_proejct/lib/python3.9/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


In [13]:
# lgb predict
y_test_pred = lgb_model.predict(test_df.drop(["target", "ID"], axis = 1))
y_test_pred_class = np.argmax(y_test_pred, axis = 1)

In [14]:
# output file 할당후 save 
submission_df = submission_df.assign(target = y_test_pred_class)
submission_df.to_csv("output.csv", index=False)